In [ ]:
""" 
Author : Hritik Jain
Date   : 16/12/2016

In this script, I'll produce a Dataset with a few images of Rs.2000 currency note
and its fake (photocopied) version.
The aim is to teach a CNN to differenciate between a fake and a real currency note, 
presumably by learning a currency note's security features.
_______________________________________________________________________________________

For all pre-processing tasks and for generating the dataset, i'll be using the OpenCV library.
"""

In [2]:
import os
import cv2
import numpy as np

In [3]:
# checking out the basic functionality

img1 = cv2.imread('Duplicate Notes 2000/Duplicate_1.jpg')
img2 = cv2.imread('Real Notes 2000/Real_1.jpg')
# converting to HSV-scale
img1 = cv2.resize(img1, (224, 224), interpolation = cv2.INTER_AREA)
img2 = cv2.resize(img2, (224, 224), interpolation = cv2.INTER_AREA)
noOfCols = 224
noOfRows = 224

cv2.imshow('img2', img1)

step = 0.030
srcPoints = np.float32([[int(step*(noOfCols-1)),int(step*(noOfRows-1))], [int((1-step)*(noOfCols-1)),int(step*(noOfRows-1))], [int(step*(noOfCols-1)),int((1-step)*(noOfRows-1))], [int((1-step)*(noOfCols-1)), int((1-step)*(noOfRows-1))]])
dstPoints = np.float32([[0,0], [noOfCols-1,0], [0,noOfRows-1], [noOfCols-1,noOfRows-1]]) 
projective_matrix = cv2.getPerspectiveTransform(srcPoints, dstPoints)
imgZoomed = cv2.warpPerspective(img1, projective_matrix, (noOfCols,noOfRows))
cv2.imwrite('zoomed.jpg', imgZoomed)

cv2.imshow('img3', imgZoomed)
cv2.waitKey()
# similarly, images can be written using the function imwrite and with the same arguments

-1

In [11]:
# for efficient memory storage :
# since the numbers are between 0 and 255, numpy's int16 datatype can be used
# secondly, we need to flatten the image array to store it efficiently ad row-major
# so for that, we will numpy's use numpy's ravel function with 'copy' flag = False
def generateDataForImages():
    """
    In this function, various techniques for augementing image data, such as random rotations
    and translations, zooming on images will performed to produce a dataset of size ~1600 images.
    RETURN :
        a list of 2-tuples where the first element in each tuple is an image's numpy 2-D array 
        and the second label is the corresponding label, 1 for real note and 0 for fake note
    """
    
    # this will be the function used to resize and grayscale the the raw input image
    def resizeGrayCenter(img):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (600, 300), interpolation = cv2.INTER_AREA)
        img = img - img.mean()
        return img
    
    # a list of 2-tuples to hold an image's numpy 2-D array 
    # and the corresponding label, 1 for real and 0 for fake
    trainingData = []
    noOfRows = 300
    noOfCols = 600
    
    # first all real currency notes' images and then all the fake ones'
    # i am skeptical of the consequences :-/
    label = 1
    
    for directory in ['Real Notes 2000/', 'Duplicate Notes 2000/']:
        for filename in os.listdir(directory):
            print filename
            img = cv2.imread(directory + filename)
            img = resizeGrayCenter(img)

            # TRANSLATIONS
            # will produce (6 + 6 + 1) x (6 + 6 + 1) = 169 images
            # stride of 5 pixels along both axis along all 4 directions
            for x in range(30, -35, -5):
                for y in range(30, -35, -5):
                    translationMatrix = np.float32([ [1,0,x], [0,1,y] ])
                    imgTrns = cv2.warpAffine(img, translationMatrix, (noOfCols, noOfRows))
                    trainingData.append((np.ravel(imgTrns.astype(np.int16, copy = False)), label))


            # ROTATIONS
            # we produce 41 different angles in the range of -10 to 10
            # with the step being equal to 0.5
            for angle in range(20, -21, -1):
                rotationMatrix = cv2.getRotationMatrix2D((noOfCols/2, noOfRows/2), float(angle)/2, 1)
                imgRotated = cv2.warpAffine(img, rotationMatrix, (noOfCols, noOfRows))
                imgRotated = perCentreMean(imgRotated)
                trainingData.append((np.ravel(imgRotated.astype(np.int16, copy = False)), label))

            # PROJECTIVE TRANSFORMATIONS for ZOOMING IN AND ZOOMING OUT
            # will produce (30 + 30) images for the dataset
            # 1ST ZOOMING IN ...
            for step in np.arange(0.001, 0.031, 0.001):
                srcPoints = np.float32([[int(step*(noOfCols-1)),int(step*(noOfRows-1))], [int((1-step)*(noOfCols-1)),int(step*(noOfRows-1))], [int(step*(noOfCols-1)),int((1-step)*(noOfRows-1))], [int((1-step)*(noOfCols-1)), int((1-step)*(noOfRows-1))]])
                dstPoints = np.float32([[0,0], [noOfCols-1,0], [0,noOfRows-1], [noOfCols-1,noOfRows-1]]) 
                projective_matrix = cv2.getPerspectiveTransform(srcPoints, dstPoints)
                imgZoomed = cv2.warpPerspective(img, projective_matrix, (noOfCols,noOfRows))
                trainingData.append((np.ravel(imgZoomed.astype(np.int16, copy = False)), label))
            # 2ND ZOOMING OUT ...
            for step in np.arange(0.001, 0.031, 0.001):
                srcPoints = np.float32(np.float32([[0,0], [noOfCols-1,0], [0,noOfRows-1], [noOfCols-1,noOfRows-1]]))
                dstPoints = np.float32([[int(step*(noOfCols-1)),int(step*(noOfRows-1))], [int((1-step)*(noOfCols-1)),int(step*(noOfRows-1))], [int(step*(noOfCols-1)),int((1-step)*(noOfRows-1))], [int((1-step)*(noOfCols-1)), int((1-step)*(noOfRows-1))]]) 
                projective_matrix = cv2.getPerspectiveTransform(srcPoints, dstPoints)
                imgZoomed = cv2.warpPerspective(img, projective_matrix, (noOfCols,noOfRows))
                trainingData.append((np.ravel(imgZoomed.astype(np.int16, copy = False)), label))
        # set label for fake images to come
        label = 0
            
    return trainingData

In [7]:
imagesAndLabels = generateDataForImages()
imagesAndLabels = 10*imagesAndLabels
# now shuffle this data so that real and fake images are scattered in the dataset
from numpy.random import shuffle
shuffle(imagesAndLabels)
len(imagesAndLabels)

Real_2.jpg
Real_1.jpg
Real_3.jpg
Duplicate_1.jpg
Duplicate_2.jpg
Duplicate_3.jpg


16200

In [14]:
# dividing in the ratio of 0.7, 0.2, 0.1 for training, validation and test
trainingData = imagesAndLabels[:11340]
validationData = imagesAndLabels[11340:14580]
testData = imagesAndLabels[14580:]

trainingInput = []
trainingOutput = []
validationInput = []
validationOutput = []
testInput = []
testOutput = []

for i in xrange(len(trainingData)):
    trainingInput.append(trainingData[i][0])
    trainingOutput.append(trainingData[i][1])

for i in xrange(len(validationData)):
    validationInput.append(validationData[i][0])
    validationOutput.append(validationData[i][1])

for i in xrange(len(testData)):
    testInput.append(testData[i][0])
    testOutput.append(testData[i][1])
    
trainData = (trainingInput, trainingOutput)
validationData = (validationInput, validationOutput)
testData = (testInput, testOutput)

dataset = (trainData, validationData, testData)
import cPickle
with open('fakeCurrencyDetectionData.pkl','wb') as fp:
    cPickle.dump(dataset,fp, protocol = 2)